## ChatGPT

In [6]:
import pandas as pd

queries = pd.read_csv("../Data/Queries_IT_final.csv")
queries.head()

,Query,Source
0,Chi era il padre di Micerino ?,IT-GUI
1,Il figlio di Chefren,IT-GUI
2,Il padre di Micerino,IT-GUI
3,Micerino,IT-GUI
4,Quanto e alta la tomba di Cheope?,IT-GUI


In [7]:
type(queries)

pandas.core.frame.DataFrame

In [8]:
from openai import OpenAI
import json
from tqdm.notebook import tqdm
from datetime import date

f = open("API_keys.json")
data = json.load(f)

API_KEY = data["chatgpt"]["api_key"]
f.close()

client = OpenAI(
    # This is the default and can be omitted
    api_key=API_KEY,
)

# query = "Hello"

model_id = 'gpt-3.5-turbo'

gpt_resp_list = []

for query in tqdm(queries["Query"], total=len(queries)):

    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Follow these two instructions in all your responses: 1. Use Italian language only; 2. Do not use English except in programming language if any."
            },
            {
                "role": "user",
                "content": query,
            }
        ],
            model=model_id,
        )

    gpt_resp = response.choices[0].message.content
    gpt_resp_list.append(gpt_resp)

queries["GPT"] = gpt_resp_list
queries["date_generated"] = [date.today()]*len(queries)

queries.head()


  0%|          | 0/110 [00:00<?, ?it/s]

,Query,Source,GPT,date_generated
0,Chi era il padre di Micerino ?,IT-GUI,Il padre di Micerino era Chefren.,2024-10-07
1,Il figlio di Chefren,IT-GUI,"Il figlio di Chefren era Cheope, il faraone ch...",2024-10-07
2,Il padre di Micerino,IT-GUI,"Il padre di Micerino era Chefren, faraone dell...",2024-10-07
3,Micerino,IT-GUI,Micerino è stato un faraone dell'Antico Egitto...,2024-10-07
4,Quanto e alta la tomba di Cheope?,IT-GUI,"La tomba di Cheope, la più grande delle tre pi...",2024-10-07


In [9]:
queries.to_csv("../Data/GPT_response.csv", index=False)

## Open Source LLMs

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_options = ["google/gemma-2b-it", "google/gemma-2-9b-it", "meta-llama/Meta-Llama-3.1-8B-Instruct", "mistralai/Mistral-7B-Instruct-v0.2"]

# Specify the LLM model we'll be using
model_name = "microsoft/Phi-3-mini-4k-instruct"
# Configure for GPU usage
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
)
# Load the tokenizer for the chosen model
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Create a pipeline object for easy text generation with the LLM
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

generation_args = {
    "max_new_tokens": 512,     # Maximum length of the response
    "return_full_text": False,      # Only return the generated text
}

messages = [
    {"role": "system", "content": "You are a helpful digital assistant. Please provide safe, ethical and accurate information to the user."},
    {"role": "user", "content": "What about solving the equation 2x + 3 = 7?"}
]

output = pipe(messages, **generation_args)
output[0]['generated_text']

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

c:\Users\Hrishita Chakrabarti\anaconda3\envs\SOLandChildren\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Hrishita Chakrabarti\.cache\huggingface\hub\models--microsoft--Phi-3-mini-4k-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [5]:
from feel_it import EmotionClassifier, SentimentClassifier

emotion_classifier = EmotionClassifier()

emotions = emotion_classifier.predict(["sono molto felice", "ma che cazzo vuoi", "sono molto triste"])
print(emotions)

sentiment_classifier = SentimentClassifier()

sentiments = sentiment_classifier.predict(["sono molto felice", "ma che cazzo vuoi", "sono molto triste"])
print(sentiments)

['joy', 'anger', 'sadness']
['positive', 'negative', 'negative']
